## streamlit_frontend.py

```
import streamlit as st
from langgraph_backend import chatbot
from langchain_core.messages import HumanMessage

# Initialize session state for chat history
if 'message_history' not in st.session_state:
    st.session_state['message_history'] = []

# Display previous messages
for message in st.session_state['message_history']:
    # Use the correct role for the UI (user vs assistant)
    role = message['role']
    with st.chat_message(role):
        st.markdown(message['content'])

# Chat Input
user_input = st.chat_input('Type here')

if user_input:
    # 1. Add User Message to History & Display it
    st.session_state['message_history'].append({'role': 'user', 'content': user_input})
    with st.chat_message('user'):
        st.markdown(user_input)

    
    # LangGraph needs a dictionary to know which "thread" (conversation) this is.
    # We use a static ID "1" for now, but you can generate unique IDs per user.
    run_config = {"configurable": {"thread_id": "1"}}

    # 2. Get Response from Chatbot
    # We pass 'run_config' instead of the Pydantic module
    response = chatbot.invoke(
        {'messages': [HumanMessage(content=user_input)]},
        config=run_config
    )

    last_msg = response['messages'][-1]
    
    # Extract the AI's reply
    if isinstance(last_msg.content, list):
        # Filter for 'text' types and join them together
        ai_message = "".join([
            block['text'] for block in last_msg.content
            if isinstance(block, dict) and block.get('type') == 'text'
        ])
    else:
        # It is already a simple string
        ai_message = last_msg.content

    
    # 3. Add AI Message to History & Display it
    # Note: We use 'assistant' role for the AI, not 'user'
    st.session_state['message_history'].append({'role': 'assistant', 'content': ai_message})
    
    with st.chat_message('assistant'):
        st.markdown(ai_message)
```

## langgraph_backend.py

```
# Authentication
import google.auth
credentials, project_id = google.auth.default()

# Import required libraries
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage
from typing import TypedDict, Literal, Annotated

# Initialize the Vertex AI Model with the dynamic projectID
model = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",
    project=project_id,
    location="global",
)

class ChatState(TypedDict):
    messages : Annotated[list[BaseMessage], add_messages]


def chat_node(state: ChatState) -> ChatState:
    # take user query from state
    messages = state['messages']

    # pass it to LLM
    response = model.invoke(messages)

    # store the received response in state
    return{'messages':[response]}


# check pointer
checkpointer = MemorySaver()

# graph
graph = StateGraph(ChatState)

# node
graph.add_node('chat_node',chat_node)

# edges
graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END)

# compile
chatbot = graph.compile(checkpointer=checkpointer)
```

## Code to run Streamlit over Google Colab
- Check the line where `target_file = "path of Streamlit application"` is mentioned and replaced that before running it.
- Run the code as it is
- Once completed, it will generate `App URL`, click on it and your Streamlit application is ready to run

In [1]:
import os
import subprocess
import time

# --- CONFIGURATION ---
target_file = "/content/streamlit_frontend.py"
# ---------------------

if not os.path.exists(target_file):
    print(f"❌ ERROR: '{target_file}' not found. Please upload it first.")
else:
    # 1. Install & Setup
    subprocess.run(["pip", "install", "-q", "streamlit"], check=True)
    if not os.path.exists("cloudflared-linux-amd64"):
        subprocess.run(["wget", "-q", "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64"], check=True)
        subprocess.run(["chmod", "+x", "cloudflared-linux-amd64"], check=True)

    # 2. Cleanup old processes
    subprocess.run(["pkill", "-f", "streamlit"])
    subprocess.run(["pkill", "-f", "cloudflared"])
    time.sleep(2)

    # 3. Run Streamlit
    print(f"Starting Streamlit with {target_file}...")
    subprocess.Popen(["streamlit", "run", target_file, "--server.port", "8501"])

    # 4. Run Tunnel
    print("Starting Cloudflare Tunnel...")
    with open("cloudflared.log", "w") as log_file:
        subprocess.Popen(["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8501"], stdout=log_file, stderr=log_file)

    # 5. Wait & Extract (Increased wait time to 10s)
    print("Waiting 10 seconds for the tunnel to initialize...")
    time.sleep(10)

    try:
        # Fixed the regex warning by using r'' for raw string
        cmd = r"grep -o 'https://.*\.trycloudflare.com' cloudflared.log | head -n 1"
        url_match = subprocess.check_output(cmd, shell=True).decode("utf-8").strip()

        if url_match:
            print("\n" + "="*40)
            print(f"🚀 Your App URL: {url_match}")
            print("="*40)
        else:
            print("⚠️ URL not found yet. Check logs manually.")
    except Exception as e:
        print(f"Error: {e}")

Starting Streamlit with /content/streamlit_frontend.py...
Starting Cloudflare Tunnel...
Waiting 10 seconds for the tunnel to initialize...

🚀 Your App URL: https://dans-entrepreneur-sagem-hrs.trycloudflare.com
